In [1]:
import signalflow as sf
from pathlib import Path
from datetime import datetime


/home/alastor/miniconda3/envs/signalflow/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'Any' is not defined

## CORE

### Raw Data Load

In [ ]:
spot_store = sf.data.raw_store.DuckDbSpotStore(
    db_path=Path("test.duckdb")
)

loader = sf.data.source.BinanceSpotLoader(
    store = spot_store
)
await loader.download(
    pairs=["BTCUSDT", "ETHUSDT", "SOLUSDT", "BNBUSDT", "XRPUSDT"],
    start=datetime(2025, 10, 1),
    end=datetime(2025, 12, 31),
)

### RawDataFactory

In [ ]:
raw_data = sf.data.RawDataFactory.from_duckdb_spot_store(
    spot_store_path=Path("test.duckdb"),
    pairs=["BTCUSDT", "ETHUSDT", "SOLUSDT"],
    start=datetime(2025, 10, 1),
    end=datetime(2025, 12, 31),
    data_types=["spot"],
)
raw_data_view = sf.core.RawDataView(raw_data)

## FEATURES

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, ClassVar
import signalflow as sf
import polars as pl


In [ ]:
# from dataclasses import dataclass, field
# from enum import Enum
# from typing import ClassVar
# import signalflow as sf
# import polars as pl
# from signalflow.core import SfComponentType, sf_component


# class RawDataType(str, Enum):
#     """Type of raw market data."""
    
#     SPOT = "spot"
#     FUTURES = "futures"
#     PERPETUAL = "perpetual"
    
#     @property
#     def columns(self) -> set[str]:
#         """Columns guaranteed to be present."""
#         base = {"pair", "timestamp", "open", "high", "low", "close", "volume"}
        
#         if self == RawDataType.FUTURES:
#             return base | {"open_interest"}
#         elif self == RawDataType.PERPETUAL:
#             return base | {"funding_rate", "open_interest"}
        
#         return base


# @dataclass
# class Feature:
#     """Base class for all features.
    
#     Two methods to implement:
#         - compute(df): all pairs, abstract for GlobalFeature/Pipeline
#         - compute_pair(df): one pair, for regular features
    
#     Attributes:
#         requires: Input column templates, e.g. ["{price_col}"]
#         outputs: Output column templates, e.g. ["rsi_{period}"]
#     """
#     component_type: ClassVar[SfComponentType] = SfComponentType.FEATURE
#     requires: ClassVar[list[str]] = []
#     outputs: ClassVar[list[str]] = []
    
#     group_col: str = "pair"
#     ts_col: str = "timestamp"
    
#     def compute(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Compute feature for all pairs. Must override."""
#         return df.group_by(self.group_col, maintain_order=True).map_groups(self.compute_pair)
    
#     def compute_pair(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Compute feature for single pair. Override for per-pair features."""
#         raise NotImplementedError(f"{self.__class__.__name__} must implement compute_pair()")
    
#     def output_cols(self, prefix: str = "") -> list[str]:
#         """Actual output column names with parameter substitution."""
#         return [f"{prefix}{tpl.format(**self.__dict__)}" for tpl in self.outputs]
    
#     def required_cols(self) -> list[str]:
#         """Actual required column names with parameter substitution."""
#         return [
#             tpl.format(**self.__dict__) if "{" in tpl else tpl 
#             for tpl in self.requires
#         ]


# @dataclass
# @sf_component(name="test_rsi")
# class RsiFeature(Feature):
#     """Relative Strength Index.
    
#     Args:
#         period: RSI period. Default: 14.
#         price_col: Price column to use. Default: "close".
    
#     Example:
#         >>> rsi = RsiFeature(period=21)
#         >>> rsi.output_cols()  # ["rsi_21"]
#     """
    
#     period: int = 14
#     price_col: str = "close"
    
#     requires = ["{price_col}"]
#     outputs = ["rsi_{period}"]
    
#     def compute(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Compute RSI for all pairs."""
#         return df.group_by(self.group_col, maintain_order=True).map_groups(self.compute_pair)
    
#     def compute_pair(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Compute RSI for single pair."""
#         col_name = f"rsi_{self.period}"
        
#         delta = pl.col(self.price_col).diff()
#         gain = pl.when(delta > 0).then(delta).otherwise(0)
#         loss = pl.when(delta < 0).then(-delta).otherwise(0)
        
#         avg_gain = gain.rolling_mean(window_size=self.period)
#         avg_loss = loss.rolling_mean(window_size=self.period)
        
#         rs = avg_gain / avg_loss
#         rsi = 100 - (100 / (1 + rs))
        
#         return df.with_columns(rsi.alias(col_name))


# @dataclass
# class GlobalFeature(Feature):
#     """Base class for features computed across all pairs.
    
#     Override compute() with custom aggregation logic.
#     """
    
#     def compute(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Must override - compute global feature across all pairs."""
#         raise NotImplementedError(f"{self.__class__.__name__} must implement compute()")


# @dataclass
# @sf_component(name="example/global_rsi")
# class GlobalMeanRsiFeature(GlobalFeature):
#     """Mean RSI across all pairs per timestamp.
    
#     1. Compute RSI per pair
#     2. Mean across all pairs at time t → global_mean_rsi
#     3. Optionally: rsi_diff = pair_rsi - global_mean_rsi
    
#     Args:
#         period: RSI period. Default: 14.
#         add_diff: Add per-pair difference column. Default: False.
#     """
    
#     period: int = 14
#     price_col: str = "close"
#     add_diff: bool = False
    
#     requires = ["{price_col}"]
#     outputs = ["global_mean_rsi_{period}"]
    
#     def output_cols(self, prefix: str = "") -> list[str]:
#         cols = [f"{prefix}global_mean_rsi_{self.period}"]
#         if self.add_diff:
#             cols.append(f"{prefix}rsi_{self.period}_diff")
#         return cols
    
#     def compute(self, df: pl.DataFrame) -> pl.DataFrame:
#         rsi_col = f"rsi_{self.period}"
#         out_col = f"global_mean_rsi_{self.period}"
        
#         has_rsi = rsi_col in df.columns
#         if not has_rsi:
#             rsi = RsiFeature(period=self.period, price_col=self.price_col)
#             df = rsi.compute(df)
        
#         mean_df = df.group_by(self.ts_col).agg(
#             pl.col(rsi_col).mean().alias(out_col)
#         )
        
#         df = df.join(mean_df, on=self.ts_col, how="left")
        
#         if self.add_diff:
#             df = df.with_columns(
#                 (pl.col(rsi_col) - pl.col(out_col)).alias(f"rsi_{self.period}_diff")
#             )
        
#         if not has_rsi:
#             df = df.drop(rsi_col)
        
#         return df
        


# @dataclass
# class OffsetFeature(Feature):
#     """Multi-timeframe feature via offset resampling.
    
#     Creates `window` parallel time series with different offsets.
#     Each offset computes features as if on `window`-minute bars.
    
#     For GRU trained on 15m but needing 1m decisions:
#     - Creates 15 parallel series (offset 0-14)
#     - At signal time t, use offset = t % 15 to get correct features
    
#     Args:
#         feature_cls: Feature class (for Kedro serialization).
#         feature_params: Parameters for feature_cls.
#         window: Aggregation window in minutes. Default: 15.
#         prefix: Prefix for output columns. Default: "{window}m_".
    
#     Example:
#         >>> offset = OffsetFeature(
#         ...     feature_cls=RsiFeature,
#         ...     feature_params={"period": 14},
#         ...     window=15,
#         ... )
#         >>> # Outputs: 15m_rsi_14, offset
#     """
    
#     feature_cls: type[Feature] = None
#     feature_params: dict = field(default_factory=dict)
#     window: int = 15
#     prefix: str | None = None
    
#     requires = ["open", "high", "low", "close", "volume", "timestamp"]
#     outputs = ["offset"]
    
#     def __post_init__(self):
#         if self.feature_cls is None:
#             raise ValueError("OffsetFeature requires 'feature_cls'")
#         self._base = self.feature_cls(**self.feature_params)
#         if self.prefix is None:
#             self.prefix = f"{self.window}m_"
    
#     def output_cols(self, prefix: str = "") -> list[str]:
#         base_cols = self._base.output_cols(prefix=f"{prefix}{self.prefix}")
#         return base_cols + [f"{prefix}offset"]
    
#     def required_cols(self) -> list[str]:
#         return ["open", "high", "low", "close", "volume", self.ts_col]
    
#     def _resample_ohlcv(self, df: pl.DataFrame, offset: int) -> pl.DataFrame:
#         """Resample 1m OHLCV to window-minute bars with given offset."""
#         df = df.with_row_index("_row_idx")
        
#         df = df.with_columns(
#             ((pl.col("_row_idx").cast(pl.Int64) - offset) // self.window).alias("_grp")
#         )
        
#         agg_exprs = [
#             pl.col(self.ts_col).last(),
#             pl.col("open").first(),
#             pl.col("high").max(),
#             pl.col("low").min(),
#             pl.col("close").last(),
#             pl.col("volume").sum(),
#         ]
#         if self.group_col in df.columns:
#             agg_exprs.append(pl.col(self.group_col).first())
        
#         return df.group_by("_grp", maintain_order=True).agg(agg_exprs)
    
#     def compute_pair(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Compute features for all offsets and merge to original timeframe."""
#         df = df.sort(self.ts_col)
#         original_len = len(df)
#         df = df.with_row_index("_orig_idx")
        
#         df = df.with_columns(
#             (pl.col("_orig_idx") % self.window).cast(pl.UInt8).alias("offset")
#         )
        
#         offset_results = []
#         for offset in range(self.window):
#             resampled = self._resample_ohlcv(df.drop(["_orig_idx", "offset"]), offset)
            
#             with_feat = self._base.compute_pair(resampled)
#             with_feat = with_feat.with_columns(pl.lit(offset).cast(pl.UInt8).alias("_offset"))
            
#             for col in self._base.output_cols():
#                 if col in with_feat.columns:
#                     with_feat = with_feat.rename({col: f"{self.prefix}{col}"})
            
#             offset_results.append(with_feat)
        
#         all_offsets = pl.concat(offset_results)
        
#         df = df.with_columns(
#             ((pl.col("_orig_idx").cast(pl.Int64) - pl.col("offset").cast(pl.Int64)) // self.window).alias("_grp")
#         )
        
#         feature_cols = [f"{self.prefix}{col}" for col in self._base.output_cols()]
        
#         result = df.join(
#             all_offsets.select(["_grp", "_offset"] + feature_cols),
#             left_on=["_grp", "offset"],
#             right_on=["_grp", "_offset"],
#             how="left",
#         )
        
#         result = result.drop(["_orig_idx", "_grp"])
#         assert len(result) == original_len
        
#         return result
    
#     def compute(self, df: pl.DataFrame) -> pl.DataFrame:
#         return df.group_by(self.group_col, maintain_order=True).map_groups(self.compute_pair)
    
#     def to_dict(self) -> dict:
#         """Serialize for Kedro."""
#         return {
#             "feature_cls": self.feature_cls.__name__,
#             "feature_params": self.feature_params,
#             "window": self.window,
#             "prefix": self.prefix,
#         }
    
#     @classmethod
#     def from_dict(cls, data: dict, registry: dict[str, type[Feature]]) -> "OffsetFeature":
#         """Deserialize from config."""
#         return cls(
#             feature_cls=registry[data["feature_cls"]],
#             feature_params=data["feature_params"],
#             window=data["window"],
#             prefix=data.get("prefix"),
#         )




# @dataclass
# class FeaturePipeline(Feature):
#     """Orchestrates multiple features with optimized execution.
    
#     Groups consecutive per-pair features into batches for single group_by.
    
#     Args:
#         features: List of features to compute.
#         raw_data_type: Type of raw data (defines available columns).
    
#     Example:
#         >>> pipeline = FeaturePipeline(
#         ...     features=[
#         ...         RsiFeature(period=14),
#         ...         SmaFeature(period=20),
#         ...         GlobalFeature(base=RsiFeature(period=14), reference_pair="BTCUSDT"),
#         ...     ],
#         ...     raw_data_type=RawDataType.SPOT,
#         ... )
#         >>> df = pipeline.run(raw_data_view)
#     """
    
#     features: list[Feature] = field(default_factory=list)
#     raw_data_type: RawDataType = RawDataType.SPOT
    
#     requires: ClassVar[list[str]] = []
    
#     def __post_init__(self):
#         if not self.features:
#             raise ValueError("FeaturePipeline requires at least one feature")
#         self._validate()
    
#     @property
#     def outputs(self) -> list[str]:
#         """Aggregated outputs from all features."""
#         result = []
#         for f in self.features:
#             result.extend(f.output_cols())
#         return result
    
#     def output_cols(self, prefix: str = "") -> list[str]:
#         return [f"{prefix}{col}" for col in self.outputs]
    
#     def _validate(self):
#         """Validate all dependencies are satisfied."""
#         available = self.raw_data_type.columns.copy()
        
#         for f in self.features:
#             required = set(f.required_cols())
#             missing = required - available
            
#             if missing:
#                 raise ValueError(
#                     f"{f.__class__.__name__} requires {missing}, "
#                     f"available: {sorted(available)}"
#                 )
            
#             available.update(f.output_cols())
    
#     def _group_into_batches(self) -> list[list[Feature]]:
#         """Group features: consecutive per-pair → batch, global → separate."""
#         batches = []
#         current_batch = []
        
#         for f in self.features:
#             is_global = isinstance(f, (GlobalFeature, GlobalMeanRsiFeature, FeaturePipeline))
            
#             if is_global:
#                 if current_batch:
#                     batches.append(current_batch)
#                     current_batch = []
#                 batches.append([f])
#             else:
#                 current_batch.append(f)
#         if current_batch:
#             batches.append(current_batch)
        
#         return batches
    
#     def _is_per_pair_batch(self, batch: list[Feature]) -> bool:
#         """Check if batch contains only per-pair features."""
#         return not any(
#             isinstance(f, (GlobalFeature, GlobalMeanRsiFeature, FeaturePipeline)) 
#             for f in batch
#         )
    
#     def compute(self, df: pl.DataFrame) -> pl.DataFrame:
#         """Compute all features with optimized batching."""
#         df = df.sort([self.group_col, self.ts_col])
        
#         batches = self._group_into_batches()
        
#         for batch in batches:
#             if self._is_per_pair_batch(batch):
#                 def apply_batch(pair_df: pl.DataFrame, features=batch) -> pl.DataFrame:
#                     for f in features:
#                         pair_df = f.compute_pair(pair_df)
#                     return pair_df
                
#                 df = df.group_by(self.group_col, maintain_order=True).map_groups(apply_batch)
#             else:
#                 for f in batch:
#                     df = f.compute(df)
        
#         return df
    
#     def run(self, raw_data_view: sf.RawDataView) -> pl.DataFrame:
#         """Entry point: load from RawDataView and compute."""
#         df = raw_data_view.to_polars(self.raw_data_type)
#         return self.compute(df)
    
#     def to_mermaid(self) -> str:
#         """Generate Mermaid diagram of feature dependencies."""
#         lines = ["graph LR"]
#         lines.append("    subgraph Input")
#         for col in sorted(self.raw_data_type.columns):
#             lines.append(f"        {col}[{col}]")
#         lines.append("    end")
        
#         for f in self.features:
#             name = f.__class__.__name__
#             if hasattr(f, 'period'):
#                 name = f"{name}_{f.period}"
            
#             for req in f.required_cols():
#                 lines.append(f"    {req} --> {name}")
#             for out in f.output_cols():
#                 lines.append(f"    {name} --> {out}[{out}]")
        
#         return "\n".join(lines)


import signalflow as sf


pipeline = FeaturePipeline(
    features=[
        sf.feature.ExampleRsiFeature(period=140),
        sf.feature.ExampleRsiFeature(period=210),
        OffsetFeature(
            feature_cls=sf.feature.ExampleRsiFeature,
            feature_params={"period": 140},
            window=60,
            prefix="ofs_",
        ),
        sf.feature.ExampleGlobalMeanRsiFeature(period=140),
        sf.feature.ExampleGlobalMeanRsiFeature(period=210),
    ],
    raw_data_type=sf.RawDataType.SPOT,
)

df = pipeline.run(raw_data_view)
import pandas as pd

pd.set_option("plotting.backend", "plotly")
df.filter(pl.col("pair") == "BTCUSDT").select(["rsi_140", "rsi_210", "global_mean_rsi_140", "global_mean_rsi_210", "ofs_rsi_140"]).to_pandas().plot()

AttributeError: module 'signalflow.feature' has no attribute 'ExampleRsiFeature'

In [ ]:
df 


pair,timestamp,open,high,low,close,volume,trades,rsi_140,rsi_210,ofs_rsi_140,offset,global_mean_rsi_140,global_mean_rsi_210
str,datetime[μs],f64,f64,f64,f64,f64,i32,f64,f64,f64,u8,f64,f64
"""BTCUSDT""",2025-10-01 00:01:00,114048.94,114100.0,114048.93,114100.0,2.0881e6,1726,null,null,null,0,null,null
"""BTCUSDT""",2025-10-01 00:02:00,114100.0,114222.95,114099.99,114172.0,2.3579e6,3957,null,null,null,1,null,null
"""BTCUSDT""",2025-10-01 00:03:00,114171.99,114182.87,114145.1,114166.99,486037.318853,1466,null,null,null,2,null,null
"""BTCUSDT""",2025-10-01 00:04:00,114167.0,114176.25,114149.09,114164.48,741897.103869,1436,null,null,null,3,null,null
"""BTCUSDT""",2025-10-01 00:05:00,114164.48,114211.4,114158.34,114211.4,328877.124806,1147,null,null,null,4,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SOLUSDT""",2025-12-30 23:56:00,125.05,125.06,125.04,125.05,37020.21681,125,58.928571,52.205128,null,1,56.443025,52.600838
"""SOLUSDT""",2025-12-30 23:57:00,125.06,125.14,125.05,125.13,91307.62098,359,59.292035,52.44898,null,2,56.770349,52.906793
"""SOLUSDT""",2025-12-30 23:58:00,125.13,125.14,125.09,125.11,524228.40961,537,60.269865,52.82631,null,3,56.740598,53.329519


In [ ]:
abdlfds;f,dlsafgmd

In [ ]:
from dataclasses import dataclass
import polars as pl

@dataclass
@sf.core.sf_component(name='custom/log_return', override=True)
class LogReturnExtractor(sf.feature.FeatureExtractor):
    """Calculates logarithmic returns: ln(Pt / Pt-1)."""
    
    price_col: str = 'close'
    out_col: str = 'log_ret'
    period: int = 1

    def compute_group(self, group_df: pl.DataFrame, data_context: dict | None) -> pl.DataFrame:
        if self.price_col not in group_df.columns:
            raise ValueError(f"Missing required column: {self.price_col}")

        log_ret = (
            pl.col(self.price_col)
            .log()
            .diff(n=self.period)
            .alias(self.out_col)
        )
        
        return group_df.with_columns(log_ret)

In [ ]:
feature_set = sf.feature.FeatureSet(
    extractors=[
        LogReturnExtractor(price_col="close", out_col="log_ret_1", period=1), 
        LogReturnExtractor(price_col="close", out_col="log_ret_2", period=2), 
        LogReturnExtractor(price_col="close", out_col="log_ret_4", period=4), 
        LogReturnExtractor(price_col="close", out_col="log_ret_8", period=8), 
        LogReturnExtractor(price_col="close", out_col="log_ret_16", period=16), 
        LogReturnExtractor(price_col="close", out_col="log_ret_32", period=32), 
    ]
)
features_df = feature_set.extract(raw_data_view)
features_df.head()


## DETECTOR

### Custom Signal Detector

In [ ]:
@dataclass
@sf.core.sf_component(name='momentum_breakout', override=True)
class MomentumDetector(sf.detector.SignalDetector):
    """Detects large price moves based on log return thresholds."""
    
    threshold: float = 0.02 
    feature_col: str = 'log_ret'
    price_col: str = 'close'

    def __post_init__(self):
        self.feature_set = sf.feature.FeatureSet(extractors=[
            LogReturnExtractor(
                price_col=self.price_col, 
                out_col=self.feature_col,
                period=1
            )
        ])

    def detect(self, features: pl.DataFrame, context: dict | None = None) -> sf.core.Signals:
        feat = pl.col(self.feature_col)
        
        signal_type = (
            pl.when(feat > self.threshold).then(pl.lit(sf.core.SignalType.RISE.value))
            .when(feat < -self.threshold).then(pl.lit(sf.core.SignalType.FALL.value))
            .otherwise(pl.lit(sf.core.SignalType.NONE.value))
            .alias('signal_type')
        )
        
        signal_val = (
            pl.when(feat > self.threshold).then(1)
            .when(feat < -self.threshold).then(-1)
            .otherwise(0)
            .alias('signal')
        )

        df = features.with_columns([signal_type, signal_val])
        
        out = df.filter(pl.col('signal_type') != sf.core.SignalType.NONE.value)
        
        return sf.core.Signals(out.select([self.pair_col, self.ts_col, 'signal_type', 'signal']))

In [ ]:

detector = MomentumDetector(threshold=0.01)

print("--- Running Pipeline ---")
signals = detector.run(raw_data_view)

print(f"\nDetected {signals.value.height} signals:")
display(signals.value)

print("\nVerifying with Source Data:")
dates_of_interest = signals.value.get_column("timestamp")
spot_df = raw_data.get("spot")

verification = spot_df.filter(pl.col("timestamp").is_in(dates_of_interest))
display(verification)

### Result ploting

In [ ]:
import plotly.graph_objects as go
import polars as pl
import plotly.io as pio

def plot_signals(raw_df: pl.DataFrame, signals_df: pl.DataFrame, pair: str = "BTCUSDT"):

    price_data = raw_df.select(["timestamp", "pair", "close"]).filter(pl.col("pair") == pair)
    sig_data = signals_df.select(["timestamp", "pair", "signal"]).filter(pl.col("pair") == pair)

    price_data = price_data.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))
    sig_data = sig_data.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))

    df_plot = price_data.sort("timestamp").to_pandas()
    signals_with_price = sig_data.join(price_data, on=["timestamp", "pair"], how="inner")
    sig_plot = signals_with_price.to_pandas()
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_plot['timestamp'], 
        y=df_plot['close'], 
        mode='lines', 
        name=f'{pair} Price',
        line=dict(color='#2E86C1', width=1.5) 
    ))

    buys = sig_plot[sig_plot['signal'] == 1]
    if not buys.empty:
        fig.add_trace(go.Scatter(
            x=buys['timestamp'],
            y=buys['close'],
            mode='markers',
            name='Buy Signal',
            marker=dict(symbol='triangle-up', size=13, color='#00CC96', line=dict(width=1, color='black'))
        ))

    sells = sig_plot[sig_plot['signal'] == -1]
    if not sells.empty:
        fig.add_trace(go.Scatter(
            x=sells['timestamp'],
            y=sells['close'],
            mode='markers',
            name='Sell Signal',
            marker=dict(symbol='triangle-down', size=13, color='#EF553B', line=dict(width=1, color='black'))
        ))

    fig.update_layout(
        title=dict(text=f'SignalFlow: {pair} Analysis', font=dict(color='black')),
        xaxis_title='Date',
        yaxis_title='Price',
        template='plotly_white', 
        hovermode='x unified',
        height=600,
        xaxis=dict(showgrid=True, gridcolor='lightgray'),
        yaxis=dict(showgrid=True, gridcolor='lightgray')
    )

    return fig

spot_df = raw_data.get("spot")
fig = plot_signals(spot_df, signals.value, pair="ETHUSDT")
fig.write_image("spot_signals.png")

## VALIDATOR

In [ ]:


from signalflow.target import FixedHorizonLabeler


labeler = FixedHorizonLabeler(
    price_col='close', 
    horizon=10, 
    include_meta=True 
)
labeled_df = labeler.compute(
    df=raw_data_view.to_polars("spot"),
    signals=signals 
)

print("Labeled Data Sample (Signals only):")
labeled_signals = labeled_df.filter(pl.col("label") != "none")
display(labeled_signals.select(["timestamp", "label", "ret"]).head())

In [ ]:
from signalflow.validator import SklearnSignalValidator

validator = SklearnSignalValidator(
    model_type='random_forest',
    model_params={'n_estimators': 100, 'max_depth': 5, 'random_state': 42}
)


features_df = detector.preprocess(raw_data_view)

split_idx = int(features_df.height * 0.8)
X_train = features_df.slice(0, split_idx)
X_test = features_df.slice(split_idx, None)

y = labeled_df.select("label")
y_train = y.slice(0, split_idx)
y_test = y.slice(split_idx, None)

print("--- Training Validator ---")
validator.fit(X_train, y_train)
print("Model trained successfully.")

In [ ]:
validated_signals = validator.validate_signals(signals, features_df)

print("\nValidated Signals (with probabilities):")
display(
    validated_signals.value
    .select([
        "timestamp", 
        "signal_type", 
        "probability_rise", 
        "probability_fall"
    ])
    .sort("probability_rise", descending=True)
    .head()
)

### Visualizing Probability

In [ ]:
import plotly.graph_objects as go
import polars as pl
import signalflow as sf 

def plot_strategy_complete(raw_df: pl.DataFrame, val_signals: sf.core.Signals, pair: str = "BTCUSDT"):
    raw_sorted = raw_df.filter(pl.col("pair") == pair).sort("timestamp")
    raw_sorted = raw_sorted.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))
    df_plot = raw_sorted.to_pandas()
    
    sig_data = val_signals.value.filter(pl.col("pair") == pair)
    sig_data = sig_data.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))
    
    merged = sig_data.join(raw_sorted.select(["timestamp", "close"]), on="timestamp", how="inner").to_pandas()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_plot['timestamp'], 
        y=df_plot['close'], 
        mode='lines', 
        name='Price', 
        line=dict(color='#2962FF', width=1.5) 
    ))


    buys = merged[merged['signal_type'] == 'rise']
    
    if not buys.empty:
        buys_noise = buys[buys['probability_rise'] < 0.5]
        if not buys_noise.empty:
            fig.add_trace(go.Scatter(
                x=buys_noise['timestamp'], y=buys_noise['close'],
                mode='markers', name='Buy (Ignored)',
                marker=dict(
                    symbol='triangle-up', 
                    size=8, 
                    color='#B0BEC5', 
                    line=dict(width=1, color='gray')
                ),
                hovertemplate="<b>Buy (Ignored)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                text=[f"{p:.2f}" for p in buys_noise['probability_rise']]
            ))

        buys_signal = buys[buys['probability_rise'] >= 0.5]
        if not buys_signal.empty:
            sizes = 12 + (buys_signal['probability_rise'] * 15)
            fig.add_trace(go.Scatter(
                x=buys_signal['timestamp'], y=buys_signal['close'],
                mode='markers', name='Buy (Active)',
                marker=dict(
                    symbol='triangle-up', 
                    size=sizes, 
                    color=buys_signal['probability_rise'],
                    colorscale=[[0, '#B9F6CA'], [1, '#00C853']], 
                    cmin=0.5, cmax=1.0, 
                    showscale=True,
                    colorbar=dict(title="Buy Conf", x=1.02, len=0.4, y=0.8),
                    line=dict(width=1, color='black')
                ),
                hovertemplate="<b>Buy (Active)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                text=[f"{p:.2f}" for p in buys_signal['probability_rise']]
            ))


    sells = merged[merged['signal_type'] == 'fall']
    
    if not sells.empty:
        sells_noise = sells[sells['probability_fall'] < 0.5]
        if not sells_noise.empty:
            fig.add_trace(go.Scatter(
                x=sells_noise['timestamp'], y=sells_noise['close'],
                mode='markers', name='Sell (Ignored)',
                marker=dict(
                    symbol='triangle-down', 
                    size=8, 
                    color='#B0BEC5', 
                    line=dict(width=1, color='gray')
                ),
                hovertemplate="<b>Sell (Ignored)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                text=[f"{p:.2f}" for p in sells_noise['probability_fall']]
            ))

        sells_signal = sells[sells['probability_fall'] >= 0.5]
        if not sells_signal.empty:
            sizes = 12 + (sells_signal['probability_fall'] * 15)
            fig.add_trace(go.Scatter(
                x=sells_signal['timestamp'], y=sells_signal['close'],
                mode='markers', name='Sell (Active)',
                marker=dict(
                    symbol='triangle-down', 
                    size=sizes, 
                    color=sells_signal['probability_fall'],
                    colorscale=[[0, '#EF9A9A'], [1, '#C62828']], 
                    cmin=0.5, cmax=1.0,
                    showscale=True,
                    colorbar=dict(title="Sell Conf", x=1.02, len=0.4, y=0.2),
                    line=dict(width=1, color='black')
                ),
                hovertemplate="<b>Sell (Active)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                text=[f"{p:.2f}" for p in sells_signal['probability_fall']]
            ))

    fig.update_layout(
        title=dict(text=f'AI Strategy Analysis: {pair}', font=dict(size=20, color='#333')),
        template='plotly_white',
        height=700,
        hovermode='x unified',
        xaxis=dict(showgrid=True, gridcolor='#eceff1'),
        yaxis=dict(showgrid=True, gridcolor='#eceff1'),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0)
    )
    fig.show()

plot_strategy_complete(raw_data_view.to_polars("spot"), validated_signals, pair="SOLUSDT")

## STRATEGY

In [ ]:
from signalflow.strategy.broker import BacktestBroker
from signalflow.strategy.broker.executor import VirtualSpotExecutor
from signalflow.data.strategy_store import DuckDbStrategyStore
from signalflow.strategy.runner import BacktestRunner, OptimizedBacktestRunner
from signalflow.strategy.component.entry import SignalEntryRule
from signalflow.strategy.component.exit import TakeProfitStopLossExit
from signalflow.analytic.strategy import TotalReturnMetric, BalanceAllocationMetric, DrawdownMetric, WinRateMetric, SharpeRatioMetric

strategy_store = DuckDbStrategyStore("strategy.duckdb")
strategy_store.init()

executor = VirtualSpotExecutor(fee_rate=0.001, slippage_pct=0.001) 
broker = BacktestBroker(executor=executor, store=strategy_store)

entry_rule = SignalEntryRule(
    base_position_size=1000.0,  
    use_probability_sizing=True, 
    min_probability=0.6,         
    max_positions_per_pair=1,
    allow_shorts=False        
)

exit_rule = TakeProfitStopLossExit(
    take_profit_pct=0.02, 
    stop_loss_pct=0.02    
)

metrics=[
        TotalReturnMetric(initial_capital=10000.0),
        BalanceAllocationMetric(initial_capital=10000.0),
        DrawdownMetric(),
        WinRateMetric(),
        SharpeRatioMetric(initial_capital=10000.0, window_size=100)
]

runner = OptimizedBacktestRunner(
    strategy_id="ml_momentum_strategy",
    broker=broker,
    entry_rules=[entry_rule],
    exit_rules=[exit_rule],
    metrics=metrics,
    initial_capital=10000.0,
    data_key="spot" 
)

print("--- Running Backtest ---")
final_state = runner.run(raw_data, validated_signals)

results = runner.get_results()
print(f"\nFinal Equity: ${results['final_equity']:.2f}")
print(f"Total Return: {results['final_return']*100:.2f}%")
print(f"Trades Executed: {results['total_trades']}")


print("Recent Trades:")
display(results['trades_df'].tail(10))

metrics_df = results['metrics_df']
metrics_df

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import polars as pl


def plot_backtest_performance(results: dict):

    metrics_df = results.get('metrics_df')
    
    if metrics_df is None or metrics_df.height == 0:
        print("No metrics to plot")
        return
    
    if 'timestamp' in metrics_df.columns:
        timestamps = metrics_df.select(
            pl.from_epoch(pl.col('timestamp').cast(pl.Int64), time_unit='s').alias('datetime')
        ).get_column('datetime').to_list()
    else:
        timestamps = list(range(metrics_df.height))
    
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.06,
        subplot_titles=(
            'Strategy Performance', 
            'Position Metrics',
            'Balance Allocation'
        ),
        row_heights=[0.45, 0.30, 0.25]
    )

    
    if 'total_return' in metrics_df.columns:
        returns_pct = (metrics_df.get_column('total_return') * 100).to_list()
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=returns_pct,
                mode='lines',
                name='Strategy Return',
                line=dict(color='#1E88E5', width=2.5),
                hovertemplate='Return: %{y:.2f}%<extra></extra>'
            ),
            row=1, col=1
        )
    
    fig.add_hline(
        y=0, 
        line_dash="dash", 
        line_color="#9E9E9E", 
        line_width=1,
        row=1, col=1
    )

    if 'open_positions' in metrics_df.columns:
        open_pos = metrics_df.get_column('open_positions').to_list()
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=open_pos,
                mode='lines',
                name='Open Positions',
                line=dict(color='#43A047', width=2),
                fill='tozeroy',
                fillcolor='rgba(67, 160, 71, 0.15)',
                hovertemplate='Open: %{y}<extra></extra>'
            ),
            row=2, col=1
        )
    
    if 'closed_positions' in metrics_df.columns:
        closed_pos = metrics_df.get_column('closed_positions').to_list()
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=closed_pos,
                mode='lines',
                name='Closed Positions',
                line=dict(color='#8E24AA', width=1.5, dash='dot'),
                hovertemplate='Closed: %{y}<extra></extra>'
            ),
            row=2, col=1
        )
    

    if 'cash' in metrics_df.columns and 'equity' in metrics_df.columns:
        cash = metrics_df.get_column('cash').to_list()
        equity = metrics_df.get_column('equity').to_list()
        
        initial_capital = results.get('initial_capital', equity[0] if equity else 10000)
        
        allocated_pct = [(eq - c) / eq if eq > 0 else 0 for eq, c in zip(equity, cash)]
        free_pct = [c / eq if eq > 0 else 0 for eq, c in zip(equity, cash)]
        
        total_balance_pct = [(eq / initial_capital - 1) * 100 for eq in equity]
        

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=free_pct,
                mode='lines',
                name='Free Cash',
                line=dict(width=0),
                fillcolor='rgba(100, 181, 246, 0.6)', 
                fill='tozeroy',
                stackgroup='balance',
                hovertemplate='Free: %{y:.1%}<extra></extra>'
            ),
            row=3, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=allocated_pct,
                mode='lines',
                name='In Positions',
                line=dict(width=0),
                fillcolor='rgba(25, 118, 210, 0.7)',  
                fill='tonexty',
                stackgroup='balance',
                hovertemplate='Allocated: %{y:.1%}<extra></extra>'
            ),
            row=3, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=total_balance_pct,
                mode='lines',
                name='Total Return',
                line=dict(color='#1565C0', width=2.5),
                yaxis='y4', 
                hovertemplate='Total: %{y:.2f}%<extra></extra>'
            ),
            row=3, col=1
        )
 
    final_return = results.get('final_return', 0) * 100
    
    fig.update_layout(
        title=dict(
            text=f'Backtest Results | Total Return: {final_return:.2f}%',
            font=dict(size=18, color='#212121')
        ),
        template='plotly_white',
        height=900,
        hovermode='x unified',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.01,
            xanchor='left',
            x=0,
            font=dict(size=11)
        ),
        showlegend=True,
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white'
    )
    
    fig.update_xaxes(title_text='Date', row=3, col=1)
    fig.update_yaxes(title_text='Return (%)', row=1, col=1)
    fig.update_yaxes(title_text='Count', row=2, col=1)
    fig.update_yaxes(
        title_text='Allocation', 
        row=3, col=1, 
        tickformat='.0%',
        range=[0, 1] 
    )
    
    fig.update_layout(
        yaxis4=dict(
            title='Total Return (%)',
            overlaying='y3',
            side='right',
            showgrid=False
        )
    )
    
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='#E0E0E0',
        showline=True,
        linewidth=1,
        linecolor='#BDBDBD'
    )
    fig.update_yaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='#E0E0E0',
        showline=True,
        linewidth=1,
        linecolor='#BDBDBD'
    )
    
    return fig


def plot_detailed_metrics(results: dict):

    metrics_df = results.get('metrics_df')
    
    if metrics_df is None or metrics_df.height == 0:
        print("No metrics to plot")
        return
    
    if 'timestamp' in metrics_df.columns:
        timestamps = metrics_df.select(
            pl.from_epoch(pl.col('timestamp').cast(pl.Int64), time_unit='s').alias('datetime')
        ).get_column('datetime').to_list()
    else:
        timestamps = list(range(metrics_df.height))
    
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=('Drawdown Analysis', 'Capital Utilization'),
        row_heights=[0.6, 0.4]
    )
    

    
    if 'current_drawdown' in metrics_df.columns:
        drawdown = metrics_df.get_column('current_drawdown').to_list()
        drawdown_pct = [-d * 100 for d in drawdown]
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=drawdown_pct,
                mode='lines',
                name='Drawdown',
                line=dict(color='#E53935', width=2),
                fill='tozeroy',
                fillcolor='rgba(229, 57, 53, 0.2)',
                hovertemplate='DD: %{y:.2f}%<extra></extra>'
            ),
            row=1, col=1
        )
        
        max_dd = results.get('max_drawdown', 0) * 100
        if max_dd > 0:
            fig.add_hline(
                y=-max_dd,
                line_dash="dash",
                line_color="#C62828",
                line_width=1.5,
                annotation_text=f"Max DD: {max_dd:.2f}%",
                annotation_position="right",
                row=1, col=1
            )
    
    
    if 'capital_utilization' in metrics_df.columns:
        util = metrics_df.get_column('capital_utilization').to_list()
        util_pct = [u * 100 for u in util]
        
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=util_pct,
                mode='lines',
                name='Capital Utilization',
                line=dict(color='#FB8C00', width=2),
                fill='tozeroy',
                fillcolor='rgba(251, 140, 0, 0.15)',
                hovertemplate='Util: %{y:.1f}%<extra></extra>'
            ),
            row=2, col=1
        )
        
        fig.add_hline(
            y=100,
            line_dash="dot",
            line_color="#9E9E9E",
            line_width=1,
            row=2, col=1
        )
    
    fig.update_layout(
        template='plotly_white',
        height=600,
        hovermode='x unified',
        showlegend=True,
        plot_bgcolor='#FAFAFA',
        paper_bgcolor='white'
    )
    
    fig.update_yaxes(title_text='Drawdown (%)', row=1, col=1)
    fig.update_yaxes(title_text='Utilization (%)', row=2, col=1)
    fig.update_xaxes(title_text='Date', row=2, col=1)
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#E0E0E0')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#E0E0E0')
    
    return fig


def print_performance_summary(results: dict):

    print("\n" + "=" * 70)
    print("📊 BACKTEST PERFORMANCE SUMMARY")
    print("=" * 70)
    
    # Performance metrics
    final_return = results.get('final_return', 0) * 100
    max_dd = results.get('max_drawdown', 0) * 100
    sharpe = results.get('sharpe_ratio', 0)
    
    print(f"\n💰 Returns:")
    print(f"   Total Return:     {final_return:>8.2f}%")
    print(f"   Max Drawdown:     {max_dd:>8.2f}%")
    print(f"   Sharpe Ratio:     {sharpe:>8.3f}")
    
    if abs(max_dd) > 0.01:
        calmar = final_return / abs(max_dd)
        print(f"   Calmar Ratio:     {calmar:>8.3f}")
    
    total_trades = results.get('total_trades', 0)
    entry_count = results.get('entry_count', 0)
    exit_count = results.get('exit_count', 0)
    win_rate = results.get('win_rate', 0) * 100 if 'win_rate' in results else 0
    
    print(f"\n📈 Trading Activity:")
    print(f"   Total Trades:     {total_trades:>8}")
    print(f"   Entry Trades:     {entry_count:>8}")
    print(f"   Exit Trades:      {exit_count:>8}")
    if win_rate > 0:
        print(f"   Win Rate:         {win_rate:>8.1f}%")
    
    final_equity = results.get('final_equity', 0)
    initial_capital = results.get('initial_capital', 10000)
    
    print(f"\n💵 Capital:")
    print(f"   Initial:          ${initial_capital:>12,.2f}")
    print(f"   Final:            ${final_equity:>12,.2f}")
    print(f"   Profit/Loss:      ${final_equity - initial_capital:>12,.2f}")
    
    print("\n" + "=" * 70 + "\n")



results = runner.get_results()

print_performance_summary(results)

fig1 = plot_backtest_performance(results)
pio.write_image(fig1, "backtest_performance.png") 

fig1.show()

fig2 = plot_detailed_metrics(results)
pio.write_image(fig2, "detailed_metrics.png") 

fig2.show()